In [1]:
import arcgis
import pandas as pd


def print_result(result):
    pd.set_option("display.max_rows", None)
    pd.set_option("display.max_colwidth", None)

    output_routes = arcgis.features.FeatureSet.from_dict(result["routes"]).sdf
    print("\n-- Output Routes -- \n")
    print(output_routes[["Name", "Total_TravelTime", "Total_Miles", "Total_Kilometers"]].to_string(index=False))

    directions = result["directions"][0]  #only for the first route
    
    print("\n-- Output Directions -- \n")
    print(f"{directions['routeName']}\n")
    for feature in directions["features"]:
        print(feature["attributes"]["text"])



stops = "-93.43656506493119,44.86055869949538;-93.47820240954265,44.85744946080268"

api_key = "AAPKb9e00fb22cea4c78b58db12e318366deonq2jiRP5icnmM8vpK86NDIPn0upOGepAiEpsjU5Wt3PEOdjjf7js9S_M-z4CmCz"
portal = arcgis.GIS("https://www.arcgis.com", api_key=api_key)
route = arcgis.network.RouteLayer(portal.properties.helperServices.route.url,
                                  gis=portal)
result = route.solve(stops=stops,
                     start_time="now",
                     return_directions=True,
                     directions_language="en")
print_result(result)



-- Output Routes -- 

                   Name  Total_TravelTime  Total_Miles  Total_Kilometers
Location 1 - Location 2          8.955083     2.591155          4.169972

-- Output Directions -- 

Location 1 - Location 2

Start at Location 1
Go west
Turn right
Turn left
At the stop sign, turn left on Technology Dr
At the traffic light, make a sharp left on Prairie Center Dr
At the traffic light, turn right on Technology Dr
Turn left on Wallace Rd
Turn right on School Rd
Bear right
Finish at Location 2, on the right


In [3]:
import json
path0 = result["routes"]["features"][0]
route_coordinates = path0["attributes"]["SHAPE"]["paths"][0]
print(json.dumps(route_coordinates, sort_keys=True, indent=4))

[
    [
        -93.43656506499997,
        44.86048000000005
    ],
    [
        -93.43698999999998,
        44.86048000000005
    ],
    [
        -93.43699999999995,
        44.86065000000008
    ],
    [
        -93.43698999999998,
        44.86081000000007
    ],
    [
        -93.43699999999995,
        44.86099000000007
    ],
    [
        -93.43699999999995,
        44.86117000000007
    ],
    [
        -93.43697999999995,
        44.861320000000035
    ],
    [
        -93.43709999999999,
        44.86142000000007
    ],
    [
        -93.43702999999994,
        44.86211000000003
    ],
    [
        -93.43903999999998,
        44.86205000000007
    ],
    [
        -93.43996999999996,
        44.861950000000036
    ],
    [
        -93.44084999999995,
        44.86167000000006
    ],
    [
        -93.44176999999996,
        44.861330000000066
    ],
    [
        -93.44198999999998,
        44.861230000000035
    ],
    [
        -93.44182999999998,
        44.86111000000

In [4]:
import arcgis
from arcgis.gis import GIS
from arcgis.features import Feature, FeatureSet
from arcgis.geometry import Polyline, Point
import arcgis.geometry

gis = GIS(api_key="AAPKb9e00fb22cea4c78b58db12e318366deonq2jiRP5icnmM8vpK86NDIPn0upOGepAiEpsjU5Wt3PEOdjjf7js9S_M-z4CmCz")

map1 = gis.map()

In [7]:
route_polyline = Polyline({"paths": [route_coordinates], "spatialReference": {"wkid": 4326}})
print(route_polyline.coordinates())
route_feature = Feature(route_polyline)
route_feature_set = FeatureSet([route_feature])
map1.add_layer(route_feature_set)

[[[-93.43656506  44.86048   ]
  [-93.43699     44.86048   ]
  [-93.437       44.86065   ]
  [-93.43699     44.86081   ]
  [-93.437       44.86099   ]
  [-93.437       44.86117   ]
  [-93.43698     44.86132   ]
  [-93.4371      44.86142   ]
  [-93.43703     44.86211   ]
  [-93.43904     44.86205   ]
  [-93.43997     44.86195   ]
  [-93.44085     44.86167   ]
  [-93.44177     44.86133   ]
  [-93.44199     44.86123   ]
  [-93.44183     44.86111   ]
  [-93.44095     44.86041   ]
  [-93.44078     44.86027   ]
  [-93.44018     44.85982   ]
  [-93.44121     44.85909   ]
  [-93.44205     44.85866   ]
  [-93.44228     44.85862   ]
  [-93.443       44.85862   ]
  [-93.4439      44.85894   ]
  [-93.44454     44.85919   ]
  [-93.44578     44.85937   ]
  [-93.44629     44.85929   ]
  [-93.44749     44.85899   ]
  [-93.44837     44.85874   ]
  [-93.44888     44.85851   ]
  [-93.4492      44.85833   ]
  [-93.44953     44.85815   ]
  [-93.45027     44.85774   ]
  [-93.45092     44.85743   ]
  [-93.451

In [5]:
crashes = [
    [-93.46523287590787, 44.85242623824904]
]


threshold_distance_meters = 100  

intersections = []

for coordinate in crashes:
    point_geometry = Point({"x": coordinate[0], "y": coordinate[1], "spatialReference": {"wkid": 4326}})

    for feature in route_feature_set.features:
        polyline = Polyline(feature.geometry)
        
        point_geometry = point_geometry.project_as(polyline.spatial_reference)


        distance = polyline.distance_to(point_geometry)
        print(distance)
        if distance is not None and distance <= threshold_distance_meters:
            intersections.append((coordinate, feature.attributes))

for coordinate, attributes in intersections:
    print(f"{coordinate} with {attributes}")

NameError: name 'route_feature_set' is not defined

In [8]:
map1


MapView(jupyter_target='notebook', layout=Layout(height='400px', width='100%'), ready=True)

In [ ]:
from arcgis.geometry import distance

point1_coords = [-118.24532, 34.05223] 
point2_coords = [-122.4194, 37.7749]

point1 = Point({"x": point1_coords[0], "y": point1_coords[1], "spatialReference": {"wkid": 4326}})
point2 = Point({"x": point2_coords[0], "y": point2_coords[1], "spatialReference": {"wkid": 4326}})

distance_meters = distance(point1.spatial_reference, [point1], point2, "aopple[aposd]")
print(distance_meters)